##### this noteboook reads the falls CID10 codes into myspace in bigquery so i can run it against tables in connectedbradford. note that csv file is in the same folder. 

In [6]:
library(bigrquery)
library(tidyverse)
data = read_csv("myocardial_infarctionicd10codes.csv")

icd10_mi <- data %>% filter(coding_system == "ICD10 codes")%>% filter(code != "I252")
icd10_mi$code <- gsub("\\.","",icd10_mi$code)

Rows: 145 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (8): code, description, coding_system, concept_id, concept_name, phenoty...
dbl (2): concept_version_id, phenotype_version_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [7]:
mybq = bq_table(project='yhcr-prd-phm-bia-core', dataset='CB_2172', table='myocardial_infarction_icd10_codes')
bq_table_upload(x=mybq, values=icd10_mi, create_disposition='CREATE_IF_NEEDED', 
                write_disposition='WRITE_TRUNCATE')

In [29]:
ProjectId = "yhcr-prd-phm-bia-core"

targetdb1 <-'yhcr-prd-phm-bia-core.CB_2172'
targetdb1 <-gsub(' ','',targetdb1)

In [30]:
sql3 <- paste('create or replace table `yhcr-prd-phm-bia-core.CB_2172.cb_2172_heart_failure` as (
with hf_aire as  
(
with episode_ch_cohort_2015 as (
select 
person_id, 
tbl_SUS_Airedale_APC_20150401_to_20190131_start_date as episode_start, 
tbl_SUS_Airedale_APC_20150401_to_20190131_end_date as episode_end, 
cast(ep_no as INT64) as episode_sequence, 
rtrim(diag1,'X') as diagnosis_1, 
rtrim(diag2,'X') as diagnosis_2,
rtrim(diag3,'X') as diagnosis_3, 
from `yhcr-prd-phm-bia-core.CB_FDM_Airedale.tbl_SUS_Airedale_APC_20150401_to_20190131`
where person_id in(select person_id from `yhcr-prd-phm-bia-core.CB_2172.temp_cb_2172_ch_cohort_demographic` )),

episode_ch_cohort_2019 as (
  select 
person_id, 
tbl_SUS_Airedale_APC_20190201_to_20200630_start_date as episode_start, 
tbl_SUS_Airedale_APC_20190201_to_20200630_end_date as episode_end, 
cast(ep_no as INT64) as episode_sequence, 
rtrim(diag1,'X') as diagnosis_1, 
rtrim(diag2,'X') as diagnosis_2,
rtrim(diag3,'X') as diagnosis_3, 
from `yhcr-prd-phm-bia-core.CB_FDM_Airedale.tbl_SUS_Airedale_APC_20190201_to_20200630`
where person_id in(select person_id from `yhcr-prd-phm-bia-core.CB_2172.temp_cb_2172_ch_cohort_demographic`)
),

episode_ch_cohort_2020 as (
  select 
person_id, 
tbl_SUS_Airedale_APC_20200701_to_20220710_mrg_start_date as episode_start, 
tbl_SUS_Airedale_APC_20200701_to_20220710_mrg_end_date as episode_end, 
cast(ep_no as INT64) as episode_sequence, 
rtrim(diag1,'X') as diagnosis_1, 
rtrim(diag2,'X') as diagnosis_2,
rtrim(diag3,'X') as diagnosis_3, 
from `yhcr-prd-phm-bia-core.CB_FDM_Airedale.tbl_SUS_Airedale_APC_20200701_to_20220710_mrg`
where person_id in(select person_id from `yhcr-prd-phm-bia-core.CB_2172.temp_cb_2172_ch_cohort_demographic`)
),
#for some reason the airedale hospital records are split into prea nd post 2019 so i had to combine the two tables toether to ensure no episodes were missed 

episode_ch_cohort_combined as (
  select
  * 
  from episode_ch_cohort_2015

  union all

  select
  * 
  from episode_ch_cohort_2019

  union all

  select
  *
  from episode_ch_cohort_2020
),


#joins care home entry date from the master cohort table 
episode_entrydate_join as (
  select 
  e_c.*,
  a.first_episodestartdate
  from episode_ch_cohort_combined as e_c
  left join `yhcr-prd-phm-bia-core.CB_2172.temp_cb_2172_ch_cohort_demographic` a
  on e_c.person_id = a.person_id),
#filters out episodes which began after the admission date to a care home
episode_ch_filtered as (
  select
e_e_j.*
  from episode_entrydate_join e_e_j
  where (episode_start > first_episodestartdate) AND (datetime_diff(episode_start,first_episodestartdate, day) < 366)
)

## up to this point reprents the code to identify and clean alll seocndary inpatient episodes for indiivudals in the connected bradford cohort in the 12 months after admission,the next line of code will identify admission related to falls, will then union this with a table for falls in AE

hf_1 as (
select
*,
#this line ranks episodes according to the date they occurred for each person, this way we can identify the most recent hospital attendance following admission 
row_number() over (partition by person_id order by episode_start) as admission_seq
from episode_ch_filtered
where diagnosis_1 in(select ICD from `yhcr-prd-phm-bia-core.CB_MYSPACE_MK.fall_icd10_code`) OR diagnosis_2 in(select ICD from `yhcr-prd-phm-bia-core.CB_MYSPACE_MK.fall_icd10_code`) OR diagnosis_3 in(select ICD from `yhcr-prd-phm-bia-core.CB_MYSPACE_MK.fall_icd10_code`)
order by person_id)
#this then filters for the first hopsital spell with a fall coded in any of the diagnosis codes. it is theoretically possible that a patient had an inpatient fall which was coded in the co-moribidities rather than the primary reason for presntation, but it is assumed the margin of error is relatively small.

select
*
from hf_1
where admission_seq = 1),

hf_bradford as (
  with episode_ch_cohort as (
select 
person_id, 
tbl_ae_start_date, 
tbl_ae_end_date, 
rtrim(diagnosis_1,'X') as diagnosis_1, 
rtrim(diagnosis_2,'X') as diagnosis_2,
rtrim(diagnosis_3,'X') as diagnosis_3, 
from `yhcr-prd-phm-bia-core.CB_FDM_BradfordRoyalInfirmary.tbl_ae`
where person_id in(select person_id from `yhcr-prd-phm-bia-core.CB_2172.temp_cb_2172_ch_cohort_demographic` )),

#joins care home entry date from the master cohort table 
episode_entrydate_join as (
  select 
  e_c.*,
  a.first_episodestartdate
  from episode_ch_cohort as e_c
  left join `yhcr-prd-phm-bia-core.CB_2172.temp_cb_2172_ch_cohort_demographic` a
  on e_c.person_id = a.person_id),
#filters out episodes which began after the admission date to a care home
episode_ch_filtered as (
  select
e_e_j.*
  from episode_entrydate_join e_e_j
  where (tbl_ae_start_date > first_episodestartdate) AND (datetime_diff(tbl_ae_start_date, first_episodestartdate, day) < 366 )
),
#filter for the first episode in a spell and extract the first three characters from each icd10 diagnosis code (not interested in the fourth character code for falls code)
episode_ch_final as (
select 
person_id, 
diagnosis_1,
diagnosis_2,
diagnosis_3,
tbl_ae_start_date,
tbl_ae_end_date,   
#substr(diagnosis_1,1,3) as diagnosis_1_substr,
#substr(diagnosis_2,1,3) as diagnosis_2_substr,
#substr(diagnosis_3,1,3) as diagnosis_3_substr,
#substr(diagnosis_4,1,3) as diagnosis_4_substr,
#substr(diagnosis_5,1,3) as diagnosis_5_substr,
#substr(diagnosis_6,1,3) as diagnosis_6_substr,
#substr(diagnosis_7,1,3) as diagnosis_7_substr,
#substr(diagnosis_8,1,3) as diagnosis_8_substr,
#substr(diagnosis_9,1,3) as diagnosis_9_substr,
#substr(diagnosis_10,1,3) as diagnosis_10_substr,
#substr(diagnosis_11,1,3) as diagnosis_11_substr,
from episode_ch_filtered
order by person_id),
#up to this point reprents the code to identify and clean all seocndary inpatient episodes for indiivudals in the connected bradford cohort in the 12 months after admission,the next line of code will identify admission related to falls, will then union this with a table for falls in AE. admissions related to falls was defined as any admission with a SNOMED OR ICD10 code available. 

hf_ch as (
select
*,
row_number() over (partition by person_id order by tbl_ae_start_date) as admission_seq
from episode_ch_final 
where diagnosis_1 in(select ICD from `yhcr-prd-phm-bia-core.CB_2172.heart_failure_icd10_codes`) OR diagnosis_2 in(select ICD from `yhcr-prd-phm-bia-core.CB_2172.heart_failure_icd10_codes`) OR diagnosis_3 in(select ICD from `yhcr-prd-phm-bia-core.CB_MYSPACE_MK.fall_icd10_code`)
order by person_id)
#this then filters for the first hopsital spell with a fall coded in any of the diagnosis codes. it is theoretically possible that a patient had an inpatient fall which was coded in the co-moribdities rather than the primary reason for presntation, but it is assumed the margin of error is relatively small.

select
*,
from fall_ch
where admission_seq = 1
order by person_id),

fall_union as (
select 
person_id,
tbl_ae_start_date
from fall_bradford


union all 

select
person_id,
fall_aire.episode_start
from
fall_aire )

select
*
from fall_union
order by person_id)
`;',sep="")

tb3 <-bq_project_query(ProjectId,sql3)

data <- bq_table_download(tb3)
data

ERROR: Error in parse(text = x, srcfile = src): <text>:10:14: unexpected symbol
9: cast(ep_no as INT64) as episode_sequence, 
10: rtrim(diag1,'X
                 ^
